<a href="https://colab.research.google.com/github/zw2788/LocalMinimaConstruction/blob/main/DwrtXGradientW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from typing import Tuple

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from IPython.display import Image

{'W_0': tensor([[ 1.0595, -0.0563],
        [-0.0375,  1.0952]], dtype=torch.float64), 'V_0': tensor([[ 3.7692],
        [-3.7214]], dtype=torch.float64), 'b': tensor([[-0.0507, -0.0689]], dtype=torch.float64), 'c': tensor([[-0.0148]], dtype=torch.float64)}


In [3]:
data = pd.read_csv(
    "https://raw.githubusercontent.com/zw2788/LocalMinimaConstruction/main/Ex1.csv")

data.head()
features = [
    "x_2dvec",

]
label = "y"

# train test split
X_raw,  Y = data[features].values, data[label].values

#convert string to array
X_raw = np.array([eval(s[0]) for s in X_raw])

# Standardize the input
# Leave blank to match the example in paper

# formatting
Y = Y.reshape((-1, 1))
print(X_raw)
print(Y)
print(X_raw.shape[0])

[[ 2.8  0.4]
 [ 3.1  4.3]
 [ 0.1 -3.4]
 [-4.2 -3.3]
 [-0.5  0.2]
 [-2.7 -0.4]
 [-3.  -4.3]
 [-0.1  3.4]
 [ 4.2  3.2]
 [ 0.4 -0.1]]
[[1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]]
10


In [20]:
class SimpleNN(nn.Module):
    def __init__(self, custom_W_0, custom_b, custom_V_0, custom_c):
        super(SimpleNN, self).__init__()

        # Ensure that the custom weights are tensors
        custom_W_0 = torch.tensor(custom_W_0, dtype=torch.float64)
        custom_b = torch.tensor(custom_b, dtype=torch.float64)
        custom_V_0 = torch.tensor(custom_V_0, dtype=torch.float64)
        custom_c = torch.tensor(custom_c, dtype=torch.float64)

        # Set the custom weights and biases
        self.W_0 = nn.Parameter(custom_W_0)
        self.b = nn.Parameter(custom_b)
        self.V_0 = nn.Parameter(custom_V_0)
        self.c = nn.Parameter(custom_c)

    def forward(self, x):
        x = F.sigmoid(torch.add(torch.matmul(x, self.W_0), self.b))
        x = F.sigmoid(torch.add(torch.matmul(x, self.V_0), self.c))
        return x

# Example usage
#custom_W_0 = [[0.1, 0.2], [0.3, 0.4]]  # Replace with your own initial values
#custom_b = [0.1, 0.2]  # Replace with your own initial values
#custom_V_0 = [[0.1], [0.2]]  # Replace with your own initial values
#custom_c = [0.1]  # Replace with your own initial values


def calculate_second_order_grad(model, X_raw_torch, Y_torch):
    # Forward pass
    output = model(X_raw_torch)
    # Compute loss
    loss = -torch.mean(Y_torch * torch.log(output) + (1 - Y_torch) * torch.log(1 - output))
    # Compute gradients of the loss w.r.t. weights
    loss.backward(create_graph=True)
    # Combine and compute the norm of all gradients
    all_grads = torch.cat([param.grad.flatten() for param in model.parameters()])
    grad_norm = torch.norm(all_grads)
    # Compute the derivative of the grad_norm with respect to X
    second_order_grad = torch.autograd.grad(grad_norm, X_raw_torch, retain_graph=True)[0]
    return second_order_grad


In [18]:
# Convert data to PyTorch tensors
X_raw_torch = torch.tensor(X_raw, requires_grad=True)
print(X_raw_torch)
Y_torch = torch.tensor(Y)

# Forward pass
output = nn_model(X_raw_torch)

# Compute loss
loss = -torch.mean(Y_torch * torch.log(output) + (1 - Y_torch) * torch.log(1 - output))
print(loss)
# Compute gradients of the loss w.r.t. weights
loss.backward(create_graph=True)


# Combine and compute the norm of all gradients
all_grads = torch.cat([nn_model.W_0.grad.flatten(), nn_model.V_0.grad.flatten(), nn_model.b.grad.flatten(), nn_model.c.grad.flatten()])
print(all_grads)
grad_norm = torch.norm(all_grads)
print(grad_norm)
# Compute the derivative of the grad_norm with respect to X
second_order_grad = torch.autograd.grad(grad_norm, X_raw_torch, retain_graph=True)[0]
print(torch.norm(second_order_grad))
# If you want to perform gradient descent on X_raw
learning_rate = 0.01
#X_raw_torch.data -= learning_rate * second_order_grad

tensor([[ 2.8000,  0.4000],
        [ 3.1000,  4.3000],
        [ 0.1000, -3.4000],
        [-4.2000, -3.3000],
        [-0.5000,  0.2000],
        [-2.7000, -0.4000],
        [-3.0000, -4.3000],
        [-0.1000,  3.4000],
        [ 4.2000,  3.2000],
        [ 0.4000, -0.1000]], dtype=torch.float64, requires_grad=True)
tensor(0.5777, dtype=torch.float64, grad_fn=<NegBackward0>)
tensor([-4.6034e-07,  7.7605e-07, -9.7695e-07,  3.9412e-07, -3.1508e-07,
        -3.0137e-07, -1.2987e-07,  1.6488e-07, -3.0719e-07],
       dtype=torch.float64, grad_fn=<CatBackward0>)
tensor(1.5008e-06, dtype=torch.float64, grad_fn=<LinalgVectorNormBackward0>)
tensor(0.1384, dtype=torch.float64)


In [19]:
# Set a threshold for the norm of the second-order gradient
threshold = 0.05 # Adjust this threshold as needed
max_iterations = 10  # Maximum number of iterations to prevent infinite loops

# Learning rate
learning_rate = 0.2

# Monte Carlo method sampling points
MC_num_samples = 10

# Surrouning points' grad
surrounding_grads = []

# parameters for the first layer
W_0 = np.array([[1.05954587,-0.05625762],[-0.03749863,1.09518945]])
b = np.array([[-0.050686,-0.06894291]])

# parameters for the second layer

V_0 = np.array([[3.76921058],[-3.72139955]])
c = np.array([[-0.0148436]])

nn_model = SimpleNN(W_0, b, V_0, c)

original_weights = {
    'W_0': nn_model.W_0.data.clone(),
    'V_0': nn_model.V_0.data.clone(),
    'b': nn_model.b.data.clone(),
    'c': nn_model.c.data.clone()
}
print(original_weights)

for i in range(max_iterations):

    # Calculate the gradient at the central point
    central_grad = calculate_second_order_grad(nn_model, X_raw_torch, Y_torch)

    # Calculate the gradient at the surrounding points by MC
    for _ in range(MC_num_samples):
      perturb_weights(nn_model, std_dev=0.01)  # Assuming perturb_weights is defined as before
      print(perturb_weights)
      grad = calculate_second_order_grad(nn_model, X_raw_torch, Y_torch)
      surrounding_grads.append(grad)
      restore_weights(nn_model, original_weights)  # Restore to central point weights


    # Combine and compute the norm of all gradients
    all_grads = torch.cat([nn_model.W_0.grad.flatten(), nn_model.V_0.grad.flatten(), nn_model.b.grad.flatten(), nn_model.c.grad.flatten()])
    grad_norm = torch.norm(all_grads)

    # Compute the derivative of the grad_norm with respect to X
    second_order_grad = torch.autograd.grad(grad_norm, X_raw_torch, retain_graph=True)[0]

    # Check if the norm of the second-order gradient is below the threshold
    if torch.norm(second_order_grad) < threshold:
        print(f"Convergence reached at iteration {i}")
        break

    # Update X_raw using gradient descent
    X_raw_torch.data -= learning_rate * second_order_grad

    # Zero out gradients for the next iteration
    nn_model.zero_grad()
    X_raw_torch.grad = None

# Print final modified data
print("Final modified X_raw:")
print(X_raw_torch)

Final modified X_raw:
tensor([[ 2.7985,  0.3892],
        [ 3.1008,  4.3033],
        [ 0.0998, -3.4012],
        [-4.1983, -3.3030],
        [-0.5101,  0.1976],
        [-2.6957, -0.4041],
        [-3.0027, -4.3008],
        [-0.1050,  3.4008],
        [ 4.1978,  3.2024],
        [ 0.3985, -0.1022]], dtype=torch.float64, requires_grad=True)
